In [ ]:
import pandas as pd

# Drop comments that are less than a certain length.
# Research Data Augmentation

## Things to Work out
- Drop Comments that are less than a certain length because they might make the model worse, basically we want comments that align with their tag


In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/suvelmuttreja/professor-nlp/main/rmpDataNew.csv")
# df.head()

#df.dropna(inplace = True)
#df = df.drop(columns=['Department', 'Helpful Rating', 'Avg Difficulty', 'Avg Rating', 'First Name', 'Last Name'])


#df.head(40)
#Dropping These Categories
df.drop('Department', axis=1, inplace=True)
df.drop('First Name', axis=1, inplace=True)
df.drop('Last Name', axis=1, inplace=True)
df.drop('Avg Difficulty', axis=1, inplace=True)
df.drop('Avg Rating', axis=1, inplace=True)
df.drop('Difficulty Rating', axis=1, inplace=True)
df = df[df['Grade'].str.len() <= 3]
df.drop('Grade', axis=1, inplace=True)
df.drop('Clarity Rating', axis=1, inplace=True)


#Drop empty rows with no rating tags

# 3. Split the 'Rating Tags' based on '--' and use one-hot encoding for each tag.
# Split the 'Rating Tags' based on '--' and use one-hot encoding for each tag (case-insensitive).
tags = df['Rating Tags'].str.lower().str.split('--', expand=True).stack()
one_hot_encoded = pd.get_dummies(tags, prefix='Tag').groupby(level=0).sum()
df = pd.concat([df, one_hot_encoded], axis=1)


# 4. Drop rows where the 'Rating Tags' column is empty.
df = df[df['Rating Tags'].notna()]



# 5. Drop rows where the string length in the 'Grade' column is longer than 3.


# 6. DROPPING COMMENTS LESS THAN LENGTH OF 100
df = df[df['Comment'].str.len() >= 100]

df.head()

,Comment,Rating Tags,Tag_accessible outside class,Tag_amazing lectures,Tag_amazing lectures,Tag_beware of pop quizzes,Tag_caring,Tag_clear grading criteria,Tag_extra credit,Tag_get ready to read,...,Tag_online savvy,Tag_participation matters,Tag_respected,Tag_skip class? you won't pass.,Tag_so many papers,Tag_test heavy,Tag_tests are tough,Tag_tests? not many,Tag_tough grader,Tag_would take again
0,Professor Admiral is a passionate teacher who ...,Get ready to read--Participation matters--Caring,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,I took Islamic Law and Society as an upper lev...,Get ready to read--Inspirational--Accessible o...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Dr.Admiral is the nicest professor I've had. S...,Get ready to read--Caring--Accessible outside ...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,She is literally so nice and understanding esp...,Get ready to read--Caring,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,I loved Dr. Admiral and would definitely take ...,Get ready to read--Participation matters--Caring,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Measuring Distribution


In [ ]:
tag_columns = [col for col in df.columns if 'Tag_' in col]
tag_distribution = df[tag_columns].sum().sort_values(ascending=False)
# print(tag_distribution)
len(tag_columns)


25

In [ ]:
df.head()

,Comment,Difficulty Rating,Grade,Clarity Rating,Rating Tags,Tag_ACCESSIBLE OUTSIDE CLASS,Tag_Accessible outside class,Tag_Amazing lectures,Tag_Amazing lectures,Tag_BEWARE OF POP QUIZZES,...,Tag_SO MANY PAPERS,Tag_Skip class? You won't pass.,Tag_So many papers,Tag_TEST HEAVY,Tag_Test heavy,Tag_Tests are tough,Tag_Tests? Not many,Tag_Tough Grader,Tag_Tough grader,Tag_Would take again
0,Professor Admiral is a passionate teacher who ...,1,A,5,Get ready to read--Participation matters--Caring,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,I took Islamic Law and Society as an upper lev...,1,A,5,Get ready to read--Inspirational--Accessible o...,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Dr.Admiral is the nicest professor I've had. S...,1,A,5,Get ready to read--Caring--Accessible outside ...,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,She is literally so nice and understanding esp...,3,A+,5,Get ready to read--Caring,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,I loved Dr. Admiral and would definitely take ...,3,A,5,Get ready to read--Participation matters--Caring,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import pandas as pd

# Assuming df is your DataFrame and tag_columns is a list of your tag columns
# Remove specific columns if they exist in the DataFrame
# columns_to_drop = ['Tag_Respected by students', 'Tag_CARES ABOUT STUDENTS']
# df.drop(columns=[col for col in columns_to_drop if col in df.columns], axis=1, inplace=True)

# Update tag_columns to ensure all columns still exist in the DataFrame
tag_columns = [col for col in tag_columns if col in df.columns]

# Set threshold for resampling
threshold = 1000

# Undersample majority classes
for tag in tag_columns:
    n_samples = int(df[tag].sum())
    if n_samples > threshold:
        # Select indices to drop
        drop_indices = df[df[tag] == 1].sample(n_samples - threshold, random_state=42).index
        # Drop selected indices
        df.drop(index=drop_indices, inplace=True)

# Oversample minority classes
for tag in tag_columns:
    n_samples = int(df[tag].sum())
    if 0 < n_samples < threshold:
        # Select samples for oversampling
        additional_samples = df[df[tag] == 1].sample(threshold - n_samples, replace=True, random_state=42)
        # Concatenate additional samples
        df = pd.concat([df, additional_samples], ignore_index=True)

# Shuffle the DataFrame
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Check and print the distribution of tags
tag_distribution = df[tag_columns].sum().sort_values(ascending=False)
print(tag_distribution)


Tag_graded by few things           2222.0
Tag_respected                      2052.0
Tag_accessible outside class       1864.0
Tag_tests are tough                1700.0
Tag_skip class? you won't pass.    1687.0
Tag_lots of homework               1648.0
Tag_tough grader                   1570.0
Tag_amazing lectures               1517.0
Tag_caring                         1451.0
Tag_clear grading criteria         1451.0
Tag_amazing lectures               1448.0
Tag_participation matters          1440.0
Tag_lecture heavy                  1417.0
Tag_gives good feedback            1396.0
Tag_beware of pop quizzes          1385.0
Tag_group projects                 1375.0
Tag_test heavy                     1365.0
Tag_get ready to read              1338.0
Tag_tests? not many                1284.0
Tag_extra credit                   1271.0
Tag_hilarious                      1202.0
Tag_online savvy                   1152.0
Tag_inspirational                  1146.0
Tag_so many papers                

In [ ]:


pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import shutil
import sys

In [ ]:

# Convert all DataFrame column names to lowercase
df.columns = df.columns.str.lower()
df.columns

Index(['comment', 'rating tags', 'tag_accessible outside class',
       'tag_amazing lectures', 'tag_amazing lectures ',
       'tag_beware of pop quizzes', 'tag_caring', 'tag_clear grading criteria',
       'tag_extra credit', 'tag_get ready to read', 'tag_gives good feedback',
       'tag_graded by few things', 'tag_group projects', 'tag_hilarious',
       'tag_inspirational', 'tag_lecture heavy', 'tag_lots of homework',
       'tag_online savvy', 'tag_participation matters', 'tag_respected',
       'tag_skip class? you won't pass.', 'tag_so many papers',
       'tag_test heavy', 'tag_tests are tough', 'tag_tests? not many',
       'tag_tough grader', 'tag_would take again'],
      dtype='object')

In [ ]:
old_target_list = ['comment', 'rating tags', 'tag_accessible outside class',
       'tag_amazing lectures', 'tag_amazing lectures ',
       'tag_beware of pop quizzes', 'tag_caring', 'tag_clear grading criteria',
       'tag_extra credit', 'tag_get ready to read', 'tag_gives good feedback',
       'tag_graded by few things', 'tag_group projects', 'tag_hilarious',
       'tag_inspirational', 'tag_lecture heavy', 'tag_lots of homework',
       'tag_online savvy', 'tag_participation matters', 'tag_respected',
       "tag_skip class? you won't pass.", 'tag_so many papers',
       'tag_test heavy', 'tag_tests are tough', 'tag_tests? not many',
       'tag_tough grader', 'tag_would take again']
# target_list = [tag for tag in old_target_list if tag in df.columns]
# print(target_list)
old_target_list = [tag.lower() for tag in old_target_list]

# Filter the tags that are present in the DataFrame columns
target_list = [tag for tag in old_target_list if tag in df.columns]

len(target_list)
df[target_list].values[:, 2:]


array([[0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, ..., 0.0, 1.0, 0.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ...,
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [0.0, 0.0, 1.0, ..., 0.0, 0.0, 0.0],
       [0.0, 1.0, 0.0, ..., 0.0, 0.0, 0.0]], dtype=object)

In [ ]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 2
LEARNING_RATE = 1e-05

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
import torch

class CustomDataSet(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, target_list):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.title = self.df['comment'].astype(str)  # Ensure comments are strings
        self.targets = self.df[target_list].values[:, 2:].astype(np.float32)  # Convert targets to float32

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = self.title[index]
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs['token_type_ids'].flatten(),
            'targets': torch.tensor(self.targets[index])
        }


In [ ]:
len(df[target_list].values[0, 2:])

25

In [ ]:
train_size = 0.8
df = df.sample(frac=train_size, random_state=200).reset_index(drop=True)
val_df = df.drop(df.index).reset_index(drop=True)

In [ ]:
train_dataset = CustomDataSet(df, tokenizer, MAX_LEN, target_list)
valid_dataset = CustomDataSet(df, tokenizer, MAX_LEN, target_list)

In [ ]:
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=TRAIN_BATCH_SIZE,
    num_workers=0
)
val_data_loader = torch.utils.data.DataLoader(
    valid_dataset,
    shuffle=False,
    batch_size=VALID_BATCH_SIZE,
    num_workers=0
)



In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [ ]:
def load_ckp(checkpoint_fpath, mode, optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['optimizer'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    valid_loss_min = checkpoint['valid_loss_min']
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    f_path = checkpoint_path
    torch.save(state, f_path)
    if is_best:
        best_fpath = best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
class BERTClass(nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained("bert-base-uncased", return_dict=True)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(768, 25)
    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert_model(input_ids, attention_mask, token_type_ids)
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output
model = BERTClass()
model.to(device)

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
from rich import print
print(model)


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (intermediate): BertIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermediate_act_fn): GELUActivation()
          )
          (output): BertOutput(
            (dense): Linear(in_features=3072, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
    )
    (pooler): BertPooler(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (activation): Tanh()
    )
  )
  (dropout): Dropout(p=0.3, inplace=False)
  (linear): Linear(in_features=768, out_features=25, bias=True)
)

In [ ]:
def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs,targets)
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)


In [ ]:
val_targets=[]
val_outputs=[]

In [ ]:
pip install torchmetrics

In [ ]:
import torchmetrics

In [ ]:
import numpy as np
import torchmetrics
import torch

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    Saves the checkpoint and optionally the best model if `is_best` is True.
    """
    torch.save(state, checkpoint_path)
    if is_best:
        torch.save(state, best_model_path)

def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, loss_fn, device, checkpoint_path, best_model_path):
    # Initialize tracker for minimum validation loss
    valid_loss_min = np.Inf

    # Initialize the MultilabelAccuracy metric and move it to the correct device
    multilabel_accuracy = torchmetrics.classification.MultilabelAccuracy(num_labels=25).to(device)

    # Move the model to the correct device
    model.to(device)

    for epoch in range(1, n_epochs + 1):
        train_loss = 0.0
        valid_loss = 0.0
        # Reset accuracy tracking for the new epoch
        multilabel_accuracy.reset()

        # Set model to training mode
        model.train()
        print(f'############# Epoch {epoch}: Training Start #############')

        # Training loop
        for batch_idx, data in enumerate(training_loader):
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['attention_mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.float)

            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update the metric
            multilabel_accuracy.update(outputs, targets)

        # Compute the average accuracy over the epoch
        epoch_accuracy = multilabel_accuracy.compute()
        print(f'Epoch {epoch} Train Acc {epoch_accuracy}')

        # Compute the average loss
        train_loss /= len(training_loader)
        print(f'############# Epoch {epoch}: Training End #############')

        # Validation phase
        print(f'############# Epoch {epoch}: Validation Start #############')
        model.eval()
        val_targets = []
        val_outputs = []

        with torch.no_grad():
            for batch_idx, data in enumerate(validation_loader):
                ids = data['input_ids'].to(device, dtype=torch.long)
                mask = data['attention_mask'].to(device, dtype=torch.long)
                token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
                targets = data['targets'].to(device, dtype=torch.float)

                outputs = model(ids, mask, token_type_ids)
                loss = loss_fn(outputs, targets)
                valid_loss += loss.item()

                # Collecting outputs for further evaluation
                val_targets.extend(targets.cpu().detach().numpy().tolist())
                val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

        # Compute the average validation loss
        valid_loss /= len(validation_loader)
        print(f'############# Epoch {epoch}: Validation End #############')
        print(f'Epoch: {epoch} \tAverage Training Loss: {train_loss:.6f} \tAverage Validation Loss: {valid_loss:.6f}')

        # Save checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        save_ckp(checkpoint, False, checkpoint_path, best_model_path)

        # Save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print(f'Validation loss decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model ...')
            save_ckp(checkpoint, True, checkpoint_path, best_model_path)
            valid_loss_min = valid_loss

        print(f'############# Epoch {epoch} Done #############\n')

    return model


In [ ]:
device

device(type='cuda')

In [ ]:
trained_model = train_model(
    n_epochs=EPOCHS,
    training_loader=train_data_loader,
    validation_loader=val_data_loader,
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn,
    device=device,
    checkpoint_path="./curr_ckpt",  # Adjust path as necessary
    best_model_path='./best.pt'     # Adjust path as necessary
)

############# Epoch 1: Training Start #############

Epoch 1 Train Acc 0.8730081915855408

############# Epoch 1: Training End #############

############# Epoch 1: Validation Start #############

############# Epoch 1: Validation End #############

...

############# Epoch 2: Validation Start #############

############# Epoch 2: Validation End #############

Epoch: 2        Average Training Loss: 0.314199         Average Validation Loss: 0.289942

Validation loss decreased (0.326592 --> 0.289942). Saving model ...

############# Epoch 2 Done #############

# Testing With Other Data

In [ ]:
df.head()

,comment,rating tags,tag_accessible outside class,tag_amazing lectures,tag_amazing lectures,tag_beware of pop quizzes,tag_caring,tag_clear grading criteria,tag_extra credit,tag_get ready to read,...,tag_online savvy,tag_participation matters,tag_respected,tag_skip class? you won't pass.,tag_so many papers,tag_test heavy,tag_tests are tough,tag_tests? not many,tag_tough grader,tag_would take again
0,King was the worst Physics professor I have ha...,Tests? Not many--GRADED BY FEW THINGS--Tests a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,If you aren't a good test taker I wouldn't rec...,Amazing lectures --Clear grading criteria--Tes...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Dr. Pan is awesome. He's the best ochem prof. ...,Respected--ACCESSIBLE OUTSIDE CLASS--Would tak...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,Dr. Park is an amazing professor. She was extr...,Clear grading criteria--Amazing lectures--Caring,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Anna is a decent neuroscience teacher. The rev...,Get ready to read--Participation matters,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
len(df.columns[2:].to_list())

25

In [ ]:
# example = df['Comment'][0]

#TESTING With OTHER DATA
example = "This professor is literally the worst professor at Georgia Tech. He was the only professor for Multi whenever I had to take him. He literally doesn't teach he only looks at power points with problems that are already done and instead of explaining concepts he says all the concepts are easy and he also has a strong accent that is hard to understand."
encodings = tokenizer.encode_plus(
            example,
            None,
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

trained_model.eval()

with torch.no_grad():
  ids = encodings['input_ids'].to(device, dtype = torch.long)
  mask = encodings['attention_mask'].to(device, dtype = torch.long)
  token_type_ids = encodings['token_type_ids'].to(device, dtype = torch.long)
  output = trained_model(ids, mask, token_type_ids)
  final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()[0]
  tags = df.columns[2:].to_list()

  tags_assigned = [tag for tag, value in zip(tags, final_output) if value > 0.2]
  print("highest probability")
  print(tags_assigned)

  print("top 3")
  top_indices = np.argsort(final_output)[-3:][::-1].tolist()

  # Get the corresponding tags
  top_tags = [tags[i] for i in top_indices]
  print(top_tags)





highest probability

['tag_graded by few things', 'tag_lots of homework', 'tag_tough grader']

top 3

['tag_tough grader', 'tag_lots of homework', 'tag_graded by few things']

# Evaluating Model And Calculating Accuracy on New Data

In [ ]:
import pandas as pd
test_df = pd.read_csv('test.csv')
test_df = test_df.drop(['Unnamed: 0'], axis=1)
test_df = test_df[test_df['Comment'].str.len() >= 200]

In [ ]:
test_df.head()

,Comment,Difficulty Rating,Grade,Clarity Rating,Rating Tags,Tag_ACCESSIBLE OUTSIDE CLASS,Tag_Accessible outside class,Tag_Amazing lectures,Tag_Amazing lectures,Tag_BEWARE OF POP QUIZZES,...,Tag_Respected,Tag_SO MANY PAPERS,Tag_Skip class? You won't pass.,Tag_So many papers,Tag_TEST HEAVY,Tag_Test heavy,Tag_Tests are tough,Tag_Tough Grader,Tag_Tough grader,Tag_Would take again
0,The most demanding course I ever took but inde...,5,B-,5,Tough Grader--Skip class? You won't pass.--ACC...,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,"Professor Richa is a great instructor, and see...",4,D,3,TEST HEAVY--Amazing lectures--Would take again,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,Incredibly energetic and cares a ton about the...,2,A-,5,Amazing lectures --Inspirational--Accessible o...,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,HWs and exams are horrendous. How can there be...,5,C,2,Skip class? You won't pass.--TEST HEAVY--LECTU...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,"Appavoo is an alright professor, and he is not...",2,B+,3,EXTRA CREDIT--Group projects--Respected,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import numpy as np
arrCol = np.array(test_df.columns[6:])

In [ ]:
def calculate_accuracy(predictions, actuals):
    correct_preds = 0
    total_tags = 0

    for (pred_id, pred_tags), (actual_id, actual_tags) in zip(predictions, actuals):
        if pred_id != actual_id:
            raise ValueError(f"Mismatched IDs: {pred_id} in predictions does not match {actual_id} in actuals")
        correct_preds += len(set(pred_tags) & set(actual_tags))
        total_tags += len(set(actual_tags))

    return correct_preds / total_tags if total_tags > 0 else 0


In [ ]:
test_df.columns[2:]

Index(['Grade', 'Clarity Rating', 'Rating Tags',
       'Tag_ACCESSIBLE OUTSIDE CLASS', 'Tag_Accessible outside class',
       'Tag_Amazing lectures', 'Tag_Amazing lectures ',
       'Tag_BEWARE OF POP QUIZZES', 'Tag_Beware of pop quizzes', 'Tag_Caring',
       'Tag_Clear grading criteria', 'Tag_EXTRA CREDIT',
       'Tag_GRADED BY FEW THINGS', 'Tag_GROUP PROJECTS',
       'Tag_Get ready to read', 'Tag_Gives good feedback',
       'Tag_Graded by few things', 'Tag_Group projects', 'Tag_Hilarious',
       'Tag_Inspirational', 'Tag_LECTURE HEAVY', 'Tag_LOTS OF HOMEWORK',
       'Tag_Lecture heavy', 'Tag_Lots of homework', 'Tag_Online Savvy',
       'Tag_Participation matters', 'Tag_Respected', 'Tag_SO MANY PAPERS',
       'Tag_Skip class? You won't pass.', 'Tag_So many papers',
       'Tag_TEST HEAVY', 'Tag_Test heavy', 'Tag_Tests are tough',
       'Tag_Tough Grader', 'Tag_Tough grader', 'Tag_Would take again'],
      dtype='object')

# Old accuracy

In [ ]:
# all_predictions = []
# all_actuals = []

# for index, row in test_df.iterrows():
#     comment = row['Comment']
#     actual_tags = [tag for tag, value in zip(test_df.columns[5:], row[1:]) if value == 1]

#     # Preprocess and tokenize the comment
#     encodings = tokenizer.encode_plus(
#         comment,
#         None,
#         add_special_tokens=True,
#         max_length=MAX_LEN,
#         padding='max_length',
#         return_token_type_ids=True,
#         truncation=True,
#         return_attention_mask=True,
#         return_tensors='pt'
#     )

#     # Make prediction
#     model.eval()
#     with torch.no_grad():
#         ids = encodings['input_ids'].to(device, dtype=torch.long)
#         mask = encodings['attention_mask'].to(device, dtype=torch.long)
#         token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
#         output = model(ids, mask, token_type_ids)
#         final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()[0]

#         tags = test_df.columns[5:].to_list()
#         predicted_tags = [tags[i] for i in np.argsort(final_output)[-3:][::-1] if final_output[i] > 0.2]

#         all_predictions.append((index, predicted_tags))
#         all_actuals.append((index, actual_tags))

# # Calculate and print the accuracy
# accuracy = calculate_accuracy(all_predictions, all_actuals)
# print(all_predictions)
# print(all_actuals)
# print('Accuracy:', accuracy)


[
    (0, ['Tag_Respected', 'Tag_Amazing lectures ', 'Tag_Caring']),
    (1, ['Tag_Caring', 'Tag_Accessible outside class', 'Tag_Respected']),
    (2, ['Tag_Caring', 'Tag_ACCESSIBLE OUTSIDE CLASS']),
    (4, ['Tag_Lots of homework', 'Tag_Would take again']),
    (5, []),
    (6, ['Tag_Lots of homework', 'Tag_Would take again']),
    (7, []),
    (8, ['Tag_Online Savvy', 'Tag_Accessible outside class']),
    (9, ['Tag_Amazing lectures ', 'Tag_Accessible outside class', 'Tag_Clear grading criteria']),
    (10, ['Tag_Lots of homework', 'Tag_So many papers']),
    (11, ['Tag_Lots of homework']),
    (12, []),
    (13, []),
    (15, ['Tag_ACCESSIBLE OUTSIDE CLASS']),
    (16, []),
    (17, ['Tag_Caring', 'Tag_Accessible outside class', 'Tag_Respected']),
    (18, []),
    (19, []),
    (20, ['Tag_ACCESSIBLE OUTSIDE CLASS']),
    (21, []),
    (22, ['Tag_Amazing lectures ']),
    (23, ['Tag_Online Savvy']),
    (24, ['Tag_Online Savvy', 'Tag_EXTRA CREDIT']),
    (25, ['Tag_Lots of homework', 'Tag_Would take again', 'Tag_So many papers']),
    (26, []),
    (27, []),
    (29, ['Tag_Amazing lectures ', 'Tag_Accessible outside class', 'Tag_Clear grading criteria']),
    (30, ['Tag_ACCESSIBLE OUTSIDE CLASS', 'Tag_Amazing lectures']),
    (31, []),
    (34, []),
    (36, ['Tag_Lots of homework', 'Tag_So many papers']),
    (37, ['Tag_Caring', 'Tag_Accessible outside class', 'Tag_Respected']),
    (40, ['Tag_Lots of homework', 'Tag_Would take again']),
    (41, ['Tag_Lots of homework', 'Tag_Would take again', 'Tag_So many papers']),
    (42, []),
    (43, ['Tag_Amazing lectures ', 'Tag_Accessible outside class']),
    (44, []),
    (45, ['Tag_Lots of homework', 'Tag_Would take again', 'Tag_So many papers']),
    (46, []),
    (48, []),
    (49, []),
    (50, []),
    (51, ['Tag_Online Savvy']),
    (52, ['Tag_Amazing lectures ']),
    (53, ['Tag_Lots of homework']),
    (55, ['Tag_Amazing lectures', 'Tag_Respected']),
    (56, []),
    (57, ['Tag_Amazing lectures ']),
    (58, ['Tag_Respected', 'Tag_Amazing lectures ', 'Tag_Caring']),
    (59, []),
    (60, ['Tag_Amazing lectures ', 'Tag_Accessible outside class', 'Tag_Respected']),
    (61, ['Tag_BEWARE OF POP QUIZZES']),
    (62, ['Tag_Lots of homework', 'Tag_Beware of pop quizzes']),
    (63, []),
    (64, []),
    (65, ['Tag_Lots of homework']),
    (66, []),
    (67, ['Tag_Amazing lectures ', 'Tag_Accessible outside class', 'Tag_Clear grading criteria']),
    (68, ['Tag_Amazing lectures ', 'Tag_Clear grading criteria']),
    (69, ['Tag_Amazing lectures ']),
    (70, ['Tag_Amazing lectures', 'Tag_Caring', 'Tag_Gives good feedback']),
    (71, []),
    (72, ['Tag_Amazing lectures ', 'Tag_Accessible outside class', 'Tag_Caring']),
    (73, ['Tag_Lots of homework', 'Tag_Would take again']),
    (74, []),
    (75, ['Tag_Lots of homework']),
    (76, ['Tag_Amazing lectures ', 'Tag_Accessible outside class', 'Tag_Clear grading criteria']),
    (77, ['Tag_Online Savvy', 'Tag_Lots of homework']),
    (78, ['Tag_Lots of homework']),
    (79, ['Tag_Lots of homework', 'Tag_So many papers', 'Tag_Would take again']),
    (80, ['Tag_Clear grading criteria', 'Tag_Amazing lectures ', 'Tag_Accessible outside class']),
    (81, []),
    (82, ['Tag_ACCESSIBLE OUTSIDE CLASS', 'Tag_Amazing lectures']),
    (83, ['Tag_Amazing lectures ']),
    (84, ['Tag_Amazing lectures', 'Tag_Respected']),
    (85, []),
    (86, ['Tag_Lots of homework', 'Tag_Would take again']),
    (87, []),
    (88, []),
    (89, ['Tag_Lots of homework']),
    (90, ['Tag_Amazing lectures ']),
    (91, ['Tag_Caring', 'Tag_Accessible outside class', 'Tag_Respected']),
    (92, []),
    (93, ['Tag_Would take again']),
    (96, []),
    (97, []),
    (98, ['Tag_GROUP PROJECTS', 'Tag_GRADED BY FEW THINGS']),
    (100, []),
    (101, ['Tag_Lots of homework', 'Tag_Would take again', 'Tag_So many papers']),
    (102, []),
    (103, ['Tag_Amazing lectures ', 'Tag_Accessible outside class']),
    (104, ['Tag_Amazing lectures ', 'Tag_Caring', 'Ta

[
    (0, ['Tag_BEWARE OF POP QUIZZES', 'Tag_Tests are tough']),
    (1, ['Tag_Caring', 'Tag_Tough grader']),
    (2, ['Tag_Beware of pop quizzes', 'Tag_Clear grading criteria', 'Tag_Lots of homework']),
    (4, ['Tag_Online Savvy', 'Tag_Tests are tough', 'Tag_Tough grader']),
    (5, ['Tag_Gives good feedback', 'Tag_LOTS OF HOMEWORK', 'Tag_TEST HEAVY']),
    (6, ['Tag_LECTURE HEAVY', 'Tag_LOTS OF HOMEWORK', 'Tag_Would take again']),
    (7, ['Tag_GRADED BY FEW THINGS', 'Tag_GROUP PROJECTS', 'Tag_Would take again']),
    (8, ['Tag_Get ready to read', 'Tag_Gives good feedback', 'Tag_TEST HEAVY']),
    (
        9,
        [
            'Tag_ACCESSIBLE OUTSIDE CLASS',
            'Tag_Beware of pop quizzes',
            'Tag_Clear grading criteria',
            'Tag_TEST HEAVY'
        ]
    ),
    (10, ['Tag_BEWARE OF POP QUIZZES', 'Tag_Participation matters']),
    (11, ['Tag_EXTRA CREDIT', 'Tag_Inspirational']),
    (12, ['Tag_Amazing lectures', 'Tag_Respected']),
    (13, ['Tag_Hilarious']),
    (15, ['Tag_Respected', 'Tag_Tests are tough', 'Tag_Would take again']),
    (16, ['Tag_BEWARE OF POP QUIZZES', 'Tag_Caring', 'Tag_Lecture heavy']),
    (17, ['Tag_Caring', 'Tag_Tough grader']),
    (18, ['Tag_Caring', 'Tag_Inspirational', 'Tag_Lots of homework']),
    (19, ['Tag_Online Savvy', 'Tag_Tests are tough']),
    (20, ['Tag_BEWARE OF POP QUIZZES', 'Tag_TEST HEAVY', 'Tag_Tough grader']),
    (21, ['Tag_Amazing lectures', 'Tag_Respected']),
    (22, ['Tag_Clear grading criteria', 'Tag_LECTURE HEAVY', "Tag_Skip class? You won't pass."]),
    (23, ['Tag_ACCESSIBLE OUTSIDE CLASS', 'Tag_Lecture heavy', 'Tag_Lots of homework', 'Tag_TEST HEAVY']),
    (24, ['Tag_Clear grading criteria', 'Tag_SO MANY PAPERS']),
    (25, ['Tag_Amazing lectures', 'Tag_Group projects', 'Tag_Online Savvy']),
    (26, ['Tag_Online Savvy', 'Tag_Tests are tough']),
    (27, ['Tag_Inspirational', 'Tag_Respected', 'Tag_SO MANY PAPERS']),
    (29, ['Tag_BEWARE OF POP QUIZZES', 'Tag_Get ready to read']),
    (30, ['Tag_Gives good feedback', 'Tag_Graded by few things', 'Tag_Online Savvy']),
    (31, ['Tag_TEST HEAVY']),
    (34, ['Tag_BEWARE OF POP QUIZZES', 'Tag_EXTRA CREDIT', 'Tag_Group projects']),
    (36, ['Tag_BEWARE OF POP QUIZZES', 'Tag_Participation matters']),
    (37, ['Tag_Caring', 'Tag_Tough grader']),
    (40, ['Tag_Amazing lectures', 'Tag_GRADED BY FEW THINGS']),
    (41, ['Tag_Amazing lectures', 'Tag_GRADED BY FEW THINGS', 'Tag_Would take again']),
    (42, ['Tag_BEWARE OF POP QUIZZES', 'Tag_EXTRA CREDIT', 'Tag_Tests are tough']),
    (43, ['Tag_Clear grading criteria']),
    (44, ['Tag_LECTURE HEAVY', 'Tag_LOTS OF HOMEWORK', 'Tag_SO MANY PAPERS']),
    (45, ['Tag_Amazing lectures', 'Tag_Hilarious', 'Tag_Test heavy']),
    (46, ['Tag_Online Savvy', 'Tag_Tests are tough']),
    (48, ['Tag_Hilarious', "Tag_Skip class? You won't pass."]),
    (49, ['Tag_Get ready to read', 'Tag_Inspirational', "Tag_Skip class? You won't pass."]),
    (50, ['Tag_Inspirational', 'Tag_SO MANY PAPERS', 'Tag_TEST HEAVY']),
    (51, ['Tag_Beware of pop quizzes', 'Tag_GRADED BY FEW THINGS', 'Tag_Get ready to read']),
    (52, ['Tag_Clear grading criteria', 'Tag_Gives good feedback', 'Tag_So many papers']),
    (53, ['Tag_EXTRA CREDIT', 'Tag_Inspirational']),
    (55, ['Tag_Caring', 'Tag_Lots of homework', 'Tag_Test heavy']),
    (56, ['Tag_BEWARE OF POP QUIZZES', 'Tag_EXTRA CREDIT', 'Tag_Group projects']),
    (57, ['Tag_Clear grading criteria', 'Tag_LECTURE HEAVY', "Tag_Skip class? You won't pass."]),
    (58, ['Tag_BEWARE OF POP QUIZZES', 'Tag_Tests are tough']),
    (59, ['Tag_Amazing lectures', 'Tag_GRADED BY FEW THINGS', 'Tag_LECTURE HEAVY', 'Tag_Tests are tough']),
    (60, ['Tag_Caring', 'Tag_Lots of homework']),
    (61, ['Tag_Amazing lectures', 'Tag_GRADED BY FEW THINGS', 'Tag_Would take again']),
    (62, ['Tag_Amazing lectures', 'Tag_So many papers', 'Tag_Tough grader']),
    (63, ['Tag_BEWARE OF POP QUIZZES', 'Tag_EXTRA CREDIT', 'Tag_Tests are tough']),
    (64

Accuracy: 0.05712090163934426

33

# New Accuracy

In [ ]:
num_labels = len(test_df.columns[5:])+1


NameError: ignored

In [ ]:
import torch
import torchmetrics
import pandas as pd

# Load your test data
test_df = pd.read_csv('test.csv')
test_df = test_df.drop(['Unnamed: 0'], axis=1)
test_df = test_df[test_df['Comment'].str.len() >= 200]

num_labels = len(test_df.columns[5:])+1

# Initialize the MultilabelAccuracy metric
accuracy_metric = torchmetrics.classification.MultilabelAccuracy(num_labels=num_labels, threshold=0.2, average='macro')
accuracy_metric = accuracy_metric.to(device)

for index, row in test_df.iterrows():
    comment = row['Comment']
    actual_tags = torch.tensor([row[5:]], dtype=torch.float).to(device)

    # Append the value of the new tag "Tag_Would take again" to the actual_tags tensor
    new_tag_value = 1  # Set the value according to your needs
    actual_tags = torch.cat((actual_tags, torch.tensor([[new_tag_value]], dtype=torch.float).to(device)), dim=1)


    # The rest of your processing and model prediction code follows...



    # Make prediction
    model.eval()
    with torch.no_grad():
        ids = encodings['input_ids'].to(device, dtype=torch.long)
        mask = encodings['attention_mask'].to(device, dtype=torch.long)
        token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
        output = model(ids, mask, token_type_ids)

        probabilities = torch.sigmoid(output).squeeze()

    # Ensure probabilities have the same number of dimensions as actual_tags
    if probabilities.ndim < actual_tags.ndim:
        probabilities = probabilities.unsqueeze(0)

    # Debugging: Print shapes to understand the mismatch
    # print(f"Shape of probabilities: {probabilities.shape}, Shape of actual_tags: {actual_tags.shape}")

    # Update the accuracy metric
    accuracy_metric.update(probabilities, actual_tags)

# Calculate final accuracy
accuracy = accuracy_metric.compute()
print('Accuracy:', accuracy.item())


Accuracy: 0.8626245856285095

# Ignore this below it's wip

In [ ]:
pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 7.9 MB/s eta 0:00:00


In [ ]:
pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.1 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from transformers import BertModel, BertTokenizer
import torchmetrics

# Define the BERTClass model
class BERTClass(nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained("bert-base-uncased", return_dict=True)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(768, 25)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert_model(input_ids, attention_mask, token_type_ids)
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

# Instantiate and load the saved model
model = BERTClass()
checkpoint = torch.load('best.pt', map_location=torch.device('cuda'))  # or 'cuda' if using GPU
model.load_state_dict(checkpoint['state_dict'])
model.eval()

# Prepare the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the test data
test_df = pd.read_csv('test.csv')
test_df = test_df.drop(['Unnamed: 0'], axis=1)
test_df = test_df[test_df['Comment'].str.len() >= 200]

# Define constants and initialize metrics
MAX_LEN = 512  # Adjust as per your model's training
num_labels = len(test_df.columns[5:]) + 1
accuracy_metric = torchmetrics.classification.MultilabelAccuracy(num_labels=num_labels, threshold=0.2, average='macro')

# Predict and calculate accuracy
for index, row in test_df.iterrows():
    comment = row['Comment']
    actual_tags = torch.tensor([row[5:]], dtype=torch.float)

    # Tokenize the comment
    encodings = tokenizer.encode_plus(
        comment,
        add_special_tokens=True,
        max_length=MAX_LEN,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    # Make prediction
    with torch.no_grad():
        ids = encodings['input_ids']
        mask = encodings['attention_mask']
        token_type_ids = encodings['token_type_ids']
        output = model(ids, mask, token_type_ids)
        probabilities = torch.sigmoid(output).squeeze()

    # Ensure probabilities have the same number of dimensions as actual_tags
    if probabilities.ndim < actual_tags.ndim:
        probabilities = probabilities.unsqueeze(0)

    # Append the value of the new tag "Tag_Would take again"
    new_tag_value = 1
    actual_tags = torch.cat((actual_tags, torch.tensor([[new_tag_value]], dtype=torch.float)), dim=1)

    # Update the accuracy metric
    accuracy_metric.update(probabilities, actual_tags)

# Calculate final accuracy
accuracy = accuracy_metric.compute()
print('Accuracy:', accuracy.item())


FileNotFoundError: ignored

In [ ]:
modelTesting = BERTClass()
model

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
torch.save(model.state_dict(), 'checking.pt')

In [ ]:
modelTesting.load_state_dict(torch.load('checking.pt'))
modelTesting.to(device)

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
import torch
import torchmetrics
import pandas as pd

# Load your test data
test_df = pd.read_csv('test.csv')
test_df = test_df.drop(['Unnamed: 0'], axis=1)
test_df = test_df[test_df['Comment'].str.len() >= 200]

num_labels = len(test_df.columns[5:])+1

# Initialize the MultilabelAccuracy metric
accuracy_metric = torchmetrics.classification.MultilabelAccuracy(num_labels=num_labels, threshold=0.2, average='macro')
accuracy_metric = accuracy_metric.to(device)

for index, row in test_df.iterrows():
    comment = row['Comment']
    actual_tags = torch.tensor([row[5:]], dtype=torch.float).to(device)

    # Append the value of the new tag "Tag_Would take again" to the actual_tags tensor
    new_tag_value = 1  # Set the value according to your needs
    actual_tags = torch.cat((actual_tags, torch.tensor([[new_tag_value]], dtype=torch.float).to(device)), dim=1)


    # The rest of your processing and model prediction code follows...



    # Make prediction
    modelTesting.eval()
    with torch.no_grad():
        ids = encodings['input_ids'].to(device, dtype=torch.long)
        mask = encodings['attention_mask'].to(device, dtype=torch.long)
        token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
        output = modelTesting(ids, mask, token_type_ids)

        probabilities = torch.sigmoid(output).squeeze()

    # Ensure probabilities have the same number of dimensions as actual_tags
    if probabilities.ndim < actual_tags.ndim:
        probabilities = probabilities.unsqueeze(0)

    # Debugging: Print shapes to understand the mismatch
    # print(f"Shape of probabilities: {probabilities.shape}, Shape of actual_tags: {actual_tags.shape}")

    # Update the accuracy metric
    accuracy_metric.update(probabilities, actual_tags)

# Calculate final accuracy
accuracy = accuracy_metric.compute()
print('Accuracy:', accuracy.item())


Accuracy: 0.8626245856285095

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import shutil

# Path to your file in Colab environment
file_path_colab = 'checking.pt'

# Destination path in Google Drive
file_path_drive = '/content/drive/My Drive/checking.pt'

# Copy the file
shutil.copy(file_path_colab, file_path_drive)


'/content/drive/My Drive/checking.pt'

###  Coding and testing the thing

In [ ]:
target_list

['Tag_ACCESSIBLE OUTSIDE CLASS',
 'Tag_Accessible outside class',
 'Tag_Amazing lectures',
 'Tag_Amazing lectures ',
 'Tag_BEWARE OF POP QUIZZES',
 'Tag_Beware of pop quizzes',
 'Tag_Caring',
 'Tag_Clear grading criteria',
 'Tag_EXTRA CREDIT',
 'Tag_GRADED BY FEW THINGS',
 'Tag_GROUP PROJECTS',
 'Tag_Get ready to read',
 'Tag_Gives good feedback',
 'Tag_Graded by few things',
 'Tag_Group projects',
 'Tag_Hilarious',
 'Tag_Inspirational',
 'Tag_LECTURE HEAVY',
 'Tag_LOTS OF HOMEWORK',
 'Tag_Lecture heavy',
 'Tag_Lots of homework',
 'Tag_Online Savvy',
 'Tag_Participation matters',
 'Tag_Respected',
 'Tag_SO MANY PAPERS',
 "Tag_Skip class? You won't pass.",
 'Tag_So many papers',
 'Tag_TEST HEAVY',
 'Tag_Test heavy',
 'Tag_Tests are tough',
 'Tag_Tests? Not many',
 'Tag_Tough Grader',
 'Tag_Tough grader',
 'Tag_Would take again']

In [ ]:
# IMPORTS


In [ ]:
class BERTClass(nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained("bert-base-uncased", return_dict=True)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(768, 34)
    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert_model(input_ids, attention_mask, token_type_ids)
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output
modelTesting = BERTClass()
modelTesting.to(device)

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
modelTesting.load_state_dict(torch.load('checking.pt'))

<All keys matched successfully>

In [ ]:
# example = df['Comment'][0]

#TESTING With OTHER DATA
def predict_labels(text):
# example = "This professor is literally the worst professor at Georgia Tech. He was the only professor for Multi whenever I had to take him. He literally doesn't teach he only looks at power points with problems that are already done and instead of explaining concepts he says all the concepts are easy and he also has a strong accent that is hard to understand."
  example = text
  encodings = tokenizer.encode_plus(
            example,
            None,
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

  model.eval()

  with torch.no_grad():
    ids = encodings['input_ids'].to(device, dtype = torch.long)
    mask = encodings['attention_mask'].to(device, dtype = torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype = torch.long)
    output = model(ids, mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()[0]
    tags = df.columns[2:].to_list()

    tags_assigned = [tag for tag, value in zip(tags, final_output) if value > 0.2]
    print("highest probability")
    print(tags_assigned)

    print("top 3")
    top_indices = np.argsort(final_output)[-3:][::-1].tolist()

    # Get the corresponding tags
    top_tags = [tags[i] for i in top_indices]
    print(top_tags)





In [ ]:
predict_labels("This professor is literally the worst professor at Georgia Tech. He was the only professor for Multi whenever I had to take him. He literally doesn't teach he only looks at power points with problems that are already done and instead of explaining concepts he says all the concepts are easy and he also has a strong accent that is hard to understand.")

highest probability

['tag_graded by few things', 'tag_lots of homework', 'tag_tough grader']

top 3

['tag_tough grader', 'tag_lots of homework', 'tag_graded by few things']

In [ ]:
# Specify the path
save_path = './TheFinalOne.pt'

# Save the model state
torch.save(model.state_dict(), save_path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
import shutil
shutil.copy('TheFinalOne.pt', '/content/drive/My Drive/')

'/content/drive/My Drive/TheFinalOne.pt'